### Fnguide에서 companyGuide 에서 재무제표 & 주식정보 확인 (크롤링에 최적화)

In [3]:
ip = '43.201.116.164'

In [2]:
# ticker_list 불러오기
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
query = """
select * from ticker
where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
"""

ticker_list = pd.read_sql(query, con=engine)
engine.dispose()

# 가장 먼저 나타나는 종목 가져오기
i = 0
ticker = ticker_list['종목코드'][i]

url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'

data = pd.read_html(url, displayed_only=False)

print(data[0].columns.tolist()) # 필요없는 부분('전년동기') 삭제 필요함.
print(data[2].columns.tolist())
print(data[4].columns.tolist())

# 필요없는 데이터 삭제후, concat
data_fs_y = pd.concat([data[0].iloc[:, ~data[0].columns.str.contains('전년동기')],
                       data[2],
                       data[4]
                    ])
data_fs_y

# IFRS 연결, 별도 column을 '계정'으로 통일시켜주자.
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})
data_fs_y

['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/06', '전년동기', '전년동기(%)']
['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/06']
['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/06']


,계정,2020/12,2021/12,2022/12,2023/06
0,매출액,8720.0,9819.0,12084.0,4938.0
1,매출원가,NaN,NaN,NaN,NaN
2,매출총이익,8720.0,9819.0,12084.0,4938.0
3,판매비와관리비계산에 참여한 계정 펼치기,8509.0,9339.0,11329.0,4578.0
4,인건비,868.0,940.0,1077.0,443.0
...,...,...,...,...,...
153,연결범위변동으로인한현금의증가,NaN,NaN,NaN,NaN
154,환율변동효과,-22.0,15.0,12.0,6.0
155,현금및현금성자산의증가,-532.0,154.0,466.0,-170.0
156,기초현금및현금성자산,1230.0,698.0,852.0,1319.0


In [4]:
# 연간 재무제표를 가져오는 과정에서 올해 것만 연말 재무제표가 아님을 알고있자. 결산월을 확인해보고 12월이 아닌 것은 빼버리자.
import requests as rq
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content)

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)
fiscal_data_text # 12라는 숫자만 출력.

# 이제 12를 제외한 나머지 숫자는 다 제거해주자. -> 즉, 연간재무제표만 뽑는 작업.
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y

,계정,2020/12,2021/12,2022/12
0,매출액,8720.0,9819.0,12084.0
1,매출원가,NaN,NaN,NaN
2,매출총이익,8720.0,9819.0,12084.0
3,판매비와관리비계산에 참여한 계정 펼치기,8509.0,9339.0,11329.0
4,인건비,868.0,940.0,1077.0
...,...,...,...,...
153,연결범위변동으로인한현금의증가,NaN,NaN,NaN
154,환율변동효과,-22.0,15.0,12.0
155,현금및현금성자산의증가,-532.0,154.0,466.0
156,기초현금및현금성자산,1230.0,698.0,852.0


In [5]:
# 추가적인 클랜징 처리 (모두 nan인 데이터 제거, 여러개의 계정명이 들어간 경우도 있음, 안중요한것들 모두 제거)
data_fs_y['계정'].value_counts(ascending=False)

# 본격적인 클랜징 과정 (함수처리)
def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    df['종목코드'] = ticker
    df['공시구분'] = frequency
    
    return df

data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
data_fs_y_clean

# 분기 데이터
data_fs_q = pd.concat(
    [data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]])
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: '계정'})
data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

# 연간 재무제표, 분기 재무제표 concat
data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
data_fs_bind

,계정,기준일,값,종목코드,공시구분
0,매출액,2020-12-31,8720.0,095570,y
1,매출총이익,2020-12-31,8720.0,095570,y
2,판매비와관리비,2020-12-31,8509.0,095570,y
3,인건비,2020-12-31,868.0,095570,y
4,유무형자산상각비,2020-12-31,148.0,095570,y
...,...,...,...,...,...
554,배당금지급(-),2023-06-30,-121.0,095570,q
556,환율변동효과,2023-06-30,2.0,095570,q
557,현금및현금성자산의증가,2023-06-30,5.0,095570,q
558,기초현금및현금성자산,2023-06-30,1143.0,095570,q


### 연간, 분기 재무제표 모두 가져오기

In [6]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

# DB 연결
engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='12345678',
                      host=f'{ip}',
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
                          select * from ticker
                          where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
                          """, con=engine)

# DB 저장 쿼리
# query = """
#     insert into kor_fs (계정, 기준일, 값, 종목코드, 공시구분)
#     values (%s,%s,%s,%s,%s) as new
#     on duplicate key update
#     값=new.값
#     """
    
# 오류 발생시 저장할 리스트 생성
error_list = []


# 재무제표 클랜징 함수
def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    df['종목코드'] = ticker
    df['공시구분'] = frequency
    
    return df


# for loop
for i in tqdm(range(0, len(ticker_list))):
    
    # 티커 선택
    ticker = ticker_list['종목코드'][i]
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        
        # url 생성
        url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        
        # 데이터 받아오기
        data = pd.read_html(url, displayed_only=False)
        
        # 연간 데이터
        data_fs_y = pd.concat([
            data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]
        ])
        data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: '계정'})
        
        # 결산년 찾기
        page_data = rq.get(url)
        page_data_html = BeautifulSoup(page_data.content, 'html.parser')
        
        fiscal_data = page_data_html.select('div.corp_group1 > h2')
        fiscal_data_text = fiscal_data[1].text
        fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)
        
        # 결산년에 해당하는 계정만 남기기
        data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') | (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
        
        # 클랜징
        data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
        
        # 분기 데이터
        data_fs_q = pd.concat([
            data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]
        ])
        data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: '계정'})
        
        data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')
        
        # 두개 합치기
        data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
        
        # 재무제표 데이터를 DB에 저장
        data_fs_bind.to_sql(name='fs_data', con=engine, index=True, if_exists='append')
        
    except:
        
        # 오류 발생시 해당 종목명을 저장하고 다음 루프로 이동
        print(ticker)
        error_list.append(ticker)
        
    # 타임슬립 적용
    time.sleep(1)
    


100%|██████████| 2372/2372 [1:17:40<00:00,  1.96s/it]


,계정,기준일,값,종목코드,공시구분
0,매출액,2020-12-31,1062.0,238490,y
1,매출원가,2020-12-31,605.0,238490,y
2,매출총이익,2020-12-31,457.0,238490,y
3,판매비와관리비,2020-12-31,208.0,238490,y
4,인건비,2020-12-31,64.0,238490,y
...,...,...,...,...,...
466,기타금융부채의감소,2023-06-30,0.0,238490,q
468,환율변동효과,2023-06-30,-1.0,238490,q
469,현금및현금성자산의증가,2023-06-30,96.0,238490,q
470,기초현금및현금성자산,2023-06-30,25.0,238490,q
